In [2]:
import torch
import pandas as pd
from torch import nn
from pathlib import Path

In [3]:
raw_dir = Path("~/datasets/nursingv1")
label_dir = Path("~/eating_json")

In [9]:
# Read session and label

session_idx = 29

df = pd.read_csv(
    Path(raw_dir, f'{session_idx}', 'raw_data.csv'), 
    header=None,
    usecols=[2,3,4]
)

In [10]:
df

,2,3,4
0,2.074219,-1.036133,9.851562
1,2.121094,-1.103516,9.835938
2,2.142578,-1.093750,9.843750
3,2.113281,-1.095703,9.828125
4,2.126953,-1.120117,9.812500
...,...,...,...
262995,-0.095764,3.072266,9.937500
262996,-0.564941,3.171875,9.492188
262997,-1.103516,2.968750,8.906250
262998,-1.328125,2.726562,8.585938


In [ ]:
# Useful Functions from old work
